# Working with the database remotely (online)

Working with PyCom remotely is **discouraged**, as it does not support many features and is slow in comparison.
A local setup requires 115 GB of disk space for the database. A tutorial for that can be found in `example.ipynb`.

If that is infeasible, or if PyCom is only used in a small-scale capacity, this is how to use it remotely.



#### Differences from local setup
There are slight differences in the API when using PyCom remotely.
* Results are paginated automatically (max 100 entries per page, or 10 if loading matrices)
* The `pyc.paginate` and `pyc.load_matrices` methods are **not** available
  * `pyc.find(..., page=1, per_page=100)` is used for pagination
  * `pyc.find(..., matrix=True)` is used for loading matrices
* The helper methods for loading additional data into the dataframe (`pyc.data.*`) are **not** available

### Initialize the PyCom class
Import the PyCom class and initialize it with `remote=True`:

In [ ]:
from pycom import PyCom, ProteinParams

pyc = PyCom(remote=True)

### Query the database

Query the database by passing a dictionary of conditions:

In [ ]:
entries = pyc.find({
    #ProteinParams.ENZYME: '3.*.*.*',
    #ProteinParams.DISEASE: 'cancer',  # string search, case-insensitive
}, page=1)

entries

In [ ]:
entries.to_csv("test.csv",index=False)

In [ ]:
import pandas as pd

In [ ]:
test_csv=pd.read_csv("test.csv")

In [ ]:
test_csv

Alternatively, query the database by passing keyword arguments:

In [ ]:
entries = pyc.find(
    cofactor='FAD',  # string search, case-insensitive
    has_ptm=True,
    has_disease=True,
    page=1
)

entries

## Supported query parameters:
* `uniprot_id`: The UniProt ID of the protein.
* `sequence`: The amino acid sequence of protein to search for. (full match)
* `min_length` / `max_length`: Min/Max number of residues in the protein.
* `min_helix` / `max_helix`: Min/Max percentage of helical structure in the protein.
* `min_turn` / `max_turn`: Min/Max percentage of turn structure in the protein.
* `min_strand` / `max_strand`: Min/Max percentage of beta strand structure in the protein.
* `organism`: Taxonomic name of the genus / species of the protein. (case-insensitive)
  * Species name or any parent taxonomic level can be used. (`pyc.get_organism_list()` for full list)
  * Surround with `:` to get precise results
    * `:homo:` returns `Homo sapiens` & `Homo sapiens neanderthalensis`)
    * `homo` also returns **homo**eomma, t**homo**mys, and *hundreds* others
* `organism_id`: Precise NCBI Taxonomy ID of the species of the protein. (prefer to use `organism` instead)
* `cath`: CATH classification of the protein (`3.40.50.360` or `3.40.*.*` or `3.*`).
* `enzyme`: Enzyme Commission number of the protein. (`1.3.1.3` or `1.3.*.*` or `1.*`).
* `has_substrate`: Whether the protein has a known substrate. (`True`/`False`)
* `has_ptm`: Whether the protein has a known post-translational modification. (`True`/`False`)
* `has_pdb`: Whether the protein has a known PDB structure. (`True`/`False`)
* `disease`: The disease associated with the protein. (name of disease, case-insensitive, e.g `cancer`)
  * Use `pyc.get_disease_list()` for full list.
  * `cancer` searches for `Ovarian cancer`, `Lung cancer`, ...
* `disease_id`: The ID of the disease associated with the protein. (`DI-02205`, get_disease_list()
* `has_disease`: Whether the protein is associated with a disease. (`True`/`False`)
* `cofactor`: The cofactor associated with the protein. (name of cofactor, case-insensitive, e.g `Zn(2+)`])
* `cofactor_id`: The ID of the cofactor associated with the protein. (`CHEBI:00001`, get_cofactor_list())
* `biological_process`: Biological process associated with the protein. (e.g `antiviral defense`, use `pyc.get_biological_process_list()` for full list)
* `cellular_component`: Cellular component associated with the protein. (e.g `nucleus`, use `pyc.get_cellular_component_list()` for full list
* `domain`: Domain associated with the protein. (e.g `zinc-finger`, use `pyc.get_domain_list()` for full list)
* `ligand`: Ligand associated with the protein. (e.g `zinc`, use `pyc.get_ligand_list()` for full list
* `molecular_function`: Molecular function associated with the protein. (e.g `antioxidant activity`, use `pyc.get_molecular_function_list()` for full list
* `ptm`: Post-translational modification associated with the protein. (e.g `phosphoprotein`, use `pyc.get_ptm_list()` for full list


### Pagination

Remote PyCom automatically paginates results. The default page size is 10 entries, but can be changed with `pyc.find(..., per_page=100)`.
The maximum page size is 100 entries, or 10 if loading matrices.

When loading more entries than the page size, just set the `page` parameter to the page number:

In [ ]:
page1 = pyc.find(max_length=20, page=1, per_page=100)  # get first 100 entries with length <= 20
page2 = pyc.find(max_length=20, page=2, per_page=100)  # get entries 101-200 with length <= 20

# pages can be concatenated
import pandas as pd
pages = pd.concat([page1, page2], ignore_index=True)

print(f'Page 1: {len(page1)} entries, Page 2: {len(page2)} entries, Total: {len(pages)} entries')

### Load coevolution matrices

Coevolution matrices can be loaded by setting the `matrix` param: `pyc.find(..., matrix=True)`.

This loads them into the `matrix` column of the dataframe.

In [ ]:
results = pyc.find(max_length=20, page=1, matrix=True)

results.iloc[0].matrix  # show the coevolution matrix for the first entry

By default, the matrices are loaded as a `numpy.ndarray`. Different formats can be specified.

Here is an example of the matrices being loaded as Pandas DataFrames and 2d-lists:

In [ ]:
from pycom import MatrixFormat

resultsPandas = pyc.find(max_length=20, page=1, matrix=True, mat_format=MatrixFormat.PANDAS)
resultsList = pyc.find(max_length=20, page=1, matrix=True, mat_format=MatrixFormat.LIST)

print(f'Pandas: {type(resultsPandas.iloc[0].matrix)}')
print(f'List: {type(resultsList.iloc[0].matrix)}')

### Load additional information

The list of cofactors, diseases, and organisms can loaded by calling:

In [ ]:
cofactors = pyc.get_cofactor_list()
diseases = pyc.get_disease_list()
organisms = pyc.get_organism_list()

cofactors